In [63]:
import pandas as pd
import numpy as np
import random
from scipy.stats import poisson

selecoes=pd.read_excel('/content/DadosCopaDoMundoQatar2022.xlsx', sheet_name='selecoes', index_col=0)
jogos=pd.read_excel('/content/DadosCopaDoMundoQatar2022.xlsx', sheet_name='jogos')

In [2]:
selecoes.head()

,Grupo,GrupoNumero,NomeEmIngles,PosiçãoRankingFIFA,ValorDeMercado,PontosRankingFIFA,Confederação,Copas,JogadorDestaque,FotoJogadorDestaque,LinkBandeiraPequena,LinkBandeiraGrande
Seleção,,,,,,,,,,,,
Catar,A,A1,Qatar,48,18.33,1442,AFC,0,Almoez Ali,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Equador,A,A2,Ecuador,44,125.80,1464,UEFA,0,Pervis Estupiñán,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Senegal,A,A3,Senegal,18,286.50,1585,CAF,0,Sadio Mané,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Holanda,A,A4,Netherlands,8,455.75,1679,UEFA,0,Virgil Van Dijk,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...
Inglaterra,B,B1,England,5,1360.00,1737,UEFA,1,Harry Kane,https://github.com/ricardorocha86/PrevisaoEspo...,https://cloudinary.fifa.com/api/v3/picture/fla...,https://flagdownload.com/wp-content/uploads/Fl...


In [3]:
jogos.head()

,data,hora,grupo,seleção1,seleção2,estádio
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR


In [4]:
np.random.poisson(lam=1,size=1)

array([0])

In [5]:
fifa=selecoes['PontosRankingFIFA'] #transformação inear de escala numerica
a, b=min(fifa), max(fifa)
fa, fb=0.15, 1 #definido melhor selecao com pontuação cerca de 6x maior que pior selecao
b1=(fb-fa)/(b-a)
b0=fb-b*b1
forca=b0+b1*fifa
forca

Seleção
Catar             0.243596
Equador           0.285618
Senegal           0.516742
Holanda           0.696292
Inglaterra        0.807079
Irã               0.467079
Estados Unidos    0.612247
País de Gales     0.511011
Argentina         0.872022
Arábia Saudita    0.232135
México            0.640899
Polônia           0.442247
França            0.860562
Dinamarca         0.669551
Tunísia           0.369663
Austrália         0.323820
Espanha           0.768876
Alemanha          0.658090
Japão             0.459438
Costa Rica        0.354382
Bélgica           0.969438
Canadá            0.304719
Marrocos          0.465169
Croácia           0.606517
Brasil            1.000000
Sérvia            0.449888
Suíça             0.585506
Camarões          0.325730
Portugal          0.696292
Gana              0.150000
Uruguai           0.623708
Coreia do Sul     0.404045
Name: PontosRankingFIFA, dtype: float64

In [6]:
def MediasPoisson(selecao1, selecao2):
  forca1=forca[selecao1]
  forca2=forca[selecao2]
  mgols=2.75 #meia de gols definida
  l1=mgols*forca1/(forca1+forca2)
  l2=mgols-l1
  return [l1, l2]

def Resultado(gols1, gols2):
  if gols1>gols2:
    resultado='V'
  elif gols2>gols1:
    resultado='D'
  else:
    resultado='E'
  return resultado

def Pontos(gols1, gols2):
  resultado=Resultado(gols1, gols2)
  if resultado=='V':
    pontos1, pontos2=3, 0
  elif resultado=='D':
    pontos1, pontos2=0, 3
  else:
    pontos1, pontos2=1, 1
  return [pontos1, pontos2, resultado]

In [7]:
MediasPoisson('Brasil', 'França')

[1.478048191315901, 1.271951808684099]

In [8]:
def Jogo(selecao1, selecao2):
  l1, l2=MediasPoisson(selecao1, selecao2)
  gols1=int(np.random.poisson(lam=l1,size=1))
  gols2=int(np.random.poisson(lam=l2,size=1))
  saldo1=gols1-gols2
  saldo2= -saldo1
  pontos1, pontos2, resultado=Pontos(gols1, gols2)
  placar='{}x{}'.format(gols1, gols2)
  return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, resultado, placar]

In [9]:
def Distribuicao(media):
  probs=[]
  for i in range(7): #probabilidade de 0 a 6 gols
    probs.append(poisson.pmf(i, media))
  probs.append(1-sum(probs)) #probabilidade de 7 ou mais gols
  return pd.Series(probs, index=['0', '1', '2', '3', '4', '5', '6', '7+'])

def ProbabilidadesPartida(selecao1, selecao2):
  l1, l2=MediasPoisson(selecao1, selecao2)
  d1, d2=Distribuicao(l1), Distribuicao(l2)
  matriz=np.outer(d1, d2) #Matriz de probabilidades
  vitoria=np.tril(matriz).sum()-np.trace(matriz) #Soma o triangulo inferior Traço é a soma da diagonal, prob empate
  derrota=np.triu(matriz).sum()-np.trace(matriz) #Soma o triangulo superior
  empate=1-(vitoria+derrota)

  probs=np.around([vitoria, empate, derrota], 3)
  probsp=[f'{100*i:.1f}%' for i in probs]

  nomes=['0', '1', '2', '3', '4', '5', '6', '7+']
  matriz=pd.DataFrame(matriz, columns=nomes, index=nomes)
  matriz.index=pd.MultiIndex.from_product([[selecao1], matriz.index])
  matriz.columns=pd.MultiIndex.from_product([[selecao2], matriz.columns])

  output={'selecao1': selecao1, 'selecao2': selecao2,
          'f1':forca[selecao1], 'f2':forca[selecao2],
          'media1':l1, 'media2':l2,
          'probabilidades':probsp, 'matriz':matriz}

  return output

In [10]:
ProbabilidadesPartida('Brasil', 'França')

{'selecao1': 'Brasil',
 'selecao2': 'França',
 'f1': 1.0,
 'f2': 0.8605617977528088,
 'media1': 1.478048191315901,
 'media2': 1.271951808684099,
 'probabilidades': ['42.0%', '25.4%', '32.6%'],
 'matriz':              França                                                    \
                   0         1         2         3         4         5   
 Brasil 0   0.063928  0.081313  0.051713  0.021926  0.006972  0.001774   
        1   0.094488  0.120185  0.076435  0.032407  0.010305  0.002622   
        2   0.069829  0.088819  0.056487  0.023950  0.007616  0.001937   
        3   0.034404  0.043760  0.027830  0.011800  0.003752  0.000955   
        4   0.012713  0.016170  0.010284  0.004360  0.001386  0.000353   
        5   0.003758  0.004780  0.003040  0.001289  0.000410  0.000104   
        6   0.000926  0.001177  0.000749  0.000318  0.000101  0.000026   
        7+  0.000239  0.000303  0.000193  0.000082  0.000026  0.000007   
 
                                    
                  

In [11]:
jogos['Vitória']=None
jogos['Empate']=None
jogos['Derrota']=None

for i in range(jogos.shape[0]):
  selecao1=jogos['seleção1'][i]
  selecao2=jogos['seleção2'][i]
  v, e, d=ProbabilidadesPartida(selecao1, selecao2)['probabilidades']
  jogos.at[i, 'Vitória']=v
  jogos.at[i, 'Empate']=e
  jogos.at[i, 'Derrota']=d
jogos.to_excel('outputEstimativasJogosCopa.xlsx')
jogos

,data,hora,grupo,seleção1,seleção2,estádio,Vitória,Empate,Derrota
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA,28.3%,24.9%,46.8%
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR,32.3%,25.4%,42.3%
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA,17.2%,22.1%,60.6%
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA,64.4%,21.0%,14.6%
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR,68.4%,19.6%,12.0%
5,2022-11-29,12:00:00,A,Equador,Senegal,INTER. KHALIFA,20.7%,23.3%,56.0%
6,2022-11-21,10:00:00,B,Inglaterra,Irã,INTER. KHALIFA,54.6%,23.6%,21.8%
7,2022-11-21,16:00:00,B,Estados Unidos,País de Gales,AL RAYYAN,43.0%,25.3%,31.7%
8,2022-11-25,07:00:00,B,País de Gales,Irã,AL RAYYAN,40.1%,25.5%,34.4%
9,2022-11-25,16:00:00,B,Inglaterra,Estados Unidos,AL KHOR,46.1%,25.0%,28.9%


Próximo passo: Simular a Copa do Mundo

In [36]:
def JogosGrupo(dados, grupo):

  times = list(dados[dados['Grupo']==grupo].index)

  time1, time2, time3, time4 = times

  jogo1= Jogo(time1, time2)
  jogo2= Jogo(time3, time4)
  jogo3= Jogo(time1, time3)
  jogo4= Jogo(time2, time4)
  jogo5= Jogo(time1, time4)
  jogo6= Jogo(time2, time3)

  pt1, pt2, pt3, pt4 = 0,0,0,0
  gp1, gp2, gp3, gp4 = 0,0,0,0
  sg1, sg2, sg3, sg4 = 0,0,0,0

  gp1, gp2, sg1, sg2, pt1, pt2 = gp1+jogo1[0], gp2+jogo1[1], sg1+jogo1[2], sg2+jogo1[3], pt1+jogo1[4], pt2+jogo1[5]
  gp3, gp4, sg3, sg4, pt3, pt4 = gp3+jogo2[0], gp4+jogo2[1], sg3+jogo2[2], sg4+jogo2[3], pt3+jogo2[4], pt4+jogo2[5]
  gp1, gp3, sg1, sg3, pt1, pt3 = gp1+jogo3[0], gp3+jogo3[1], sg1+jogo3[2], sg3+jogo3[3], pt1+jogo3[4], pt3+jogo3[5]
  gp2, gp4, sg2, sg4, pt2, pt4 = gp2+jogo4[0], gp4+jogo4[1], sg2+jogo4[2], sg4+jogo4[3], pt2+jogo4[4], pt4+jogo4[5]
  gp1, gp4, sg1, sg4, pt1, pt4 = gp1+jogo5[0], gp4+jogo5[1], sg1+jogo5[2], sg4+jogo5[3], pt1+jogo5[4], pt4+jogo5[5]
  gp2, gp3, sg2, sg3, pt2, pt3 = gp2+jogo6[0], gp3+jogo6[1], sg2+jogo6[2], sg3+jogo6[3], pt2+jogo6[4], pt3+jogo6[5]

  tab = pd.DataFrame([[pt1, pt2, pt3, pt4], [sg1, sg2, sg3, sg4], [gp1, gp2, gp3, gp4]], columns=times, index=['Pontos', 'Saldo de Gols', 'Gols Pró']).T
  tab=tab.sort_values(by=['Pontos', 'Saldo de Gols', 'Gols Pró'], ascending=False)
  tab['Posição']=[1,2,3,4]

  partidas = [time1 + ' x '+ time2,
              time3 + ' x '+ time4,
              time1 + ' x '+ time3,
              time2 + ' x '+ time4,
              time1 + ' x '+ time4,
              time2 + ' x '+ time3,]
  resultados = [jogo1[6], jogo2[6], jogo3[6], jogo4[6], jogo5[6], jogo6[6]]
  placares = [jogo1[-1], jogo2[-1], jogo3[-1], jogo4[-1], jogo5[-1], jogo6[-1]]

  jogos=pd.DataFrame([partidas, placares, resultados]).transpose()
  jogos.columns = ['Partida', 'Placar', 'Resultado']

  return [tab, jogos]

In [39]:
grupo=JogosGrupo(selecoes, 'A')
grupo[0]

,Pontos,Saldo de Gols,Gols Pró,Posição
Equador,4,0,6,1
Catar,4,0,5,2
Senegal,4,0,4,3
Holanda,3,0,4,4


In [40]:
grupo[1]

,Partida,Placar,Resultado
0,Catar x Equador,3x2,V
1,Senegal x Holanda,1x1,E
2,Catar x Senegal,1x2,D
3,Equador x Holanda,2x2,E
4,Catar x Holanda,1x1,E
5,Equador x Senegal,2x1,V


Segunda fase da competição

In [64]:
def JogoMataMata(selecao1, selecao2):
  jogo=Jogo(selecao1, selecao2)
  resultado=jogo[6]
  if resultado == 'V':
    return selecao1
  elif resultado == 'D':
    return selecao2
  else:
    return random.sample([selecao1, selecao2],1)[0] # se empataram times tem 50% de chance de ganhar, cada

In [78]:
JogoMataMata('Brasil', 'França')

'Brasil'

In [81]:
def SimulaCopa(selecoes):
  cols = ['1st','2nd','3rd','4th','Oitavas','Quartas','Semis','Final','Campeão']
  n=selecoes.shape[0]
  m=len(cols)
  aux=np.array(np.zeros(n*m).reshape(n,m))
  info=pd.DataFrame(aux, columns=cols, index=selecoes.index)
  info=info.astype(int)

  top16=[]
  for i in list('ABCDEFGH'):
    a=JogosGrupo(selecoes, grupo=i)[0]
    top16+=a.index[:2].tolist() # Pega o primeiro e segundo colocado de cada grupo
    anomes=a.index.tolist()
    info.at[anomes[0], '1st']=1
    info.at[anomes[1], '2nd']=1
    info.at[anomes[2], '3rd']=1
    info.at[anomes[3], '4th']=1

  qf1=JogoMataMata(top16[0], top16[3])
  qf2=JogoMataMata(top16[2], top16[1])
  qf3=JogoMataMata(top16[4], top16[7])
  qf4=JogoMataMata(top16[6], top16[5])
  qf5=JogoMataMata(top16[8], top16[11])
  qf6=JogoMataMata(top16[10], top16[9])
  qf7=JogoMataMata(top16[12], top16[15])
  qf8=JogoMataMata(top16[14], top16[13])

  top8=[qf1, qf2, qf3, qf4, qf5, qf6, qf7, qf8]

  sf1=JogoMataMata(qf1, qf3)
  sf2=JogoMataMata(qf2, qf4)
  sf3=JogoMataMata(qf5, qf7)
  sf4=JogoMataMata(qf6, qf8)

  top4=[sf1, sf2, sf3, sf4]

  f1=JogoMataMata(sf1, sf3)
  f2=JogoMataMata(sf2, sf4)

  top2=[f1, f2]

  top1= JogoMataMata(f1,f2)

  info.at[top16, 'Oitavas']=1
  info.at[top8, 'Quartas']=1
  info.at[top4, 'Semis']=1
  info.at[top2, 'Final']=1
  info.at[top1, 'Campeão']=1

  return info

In [82]:
SimulaCopa(selecoes)

,1st,2nd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Catar,0,0,0,1,0,0,0,0,0
Equador,0,0,1,0,0,0,0,0,0
Senegal,1,0,0,0,1,1,0,0,0
Holanda,0,1,0,0,1,1,0,0,0
Inglaterra,0,1,0,0,1,0,0,0,0
Irã,0,0,1,0,0,0,0,0,0
Estados Unidos,0,0,0,1,0,0,0,0,0
País de Gales,1,0,0,0,1,0,0,0,0
Argentina,1,0,0,0,1,0,0,0,0


In [94]:
def SimulacaoTotal(dados, S=1000):
  print('IA: "Iniciando simulação..."')
  info=SimulaCopa(dados)
  for i in range(S-1):
    info+=SimulaCopa(dados)
    if (i+2)%(S/10)==0:
      print('IA: "Simulações de Copas do Mundo: {:.0f}% completas"'.format(100*((i+2)/S)))
  print('IA: "Simulação Finalizada!"')
  return info.sort_values(by='Campeão', ascending=False)/S

In [93]:
S=1000
sim=SimulacaoTotal(selecoes, S)
sim.to_excel('outputSimulaçõesCopaDoMundo.xlsx')
sim

IA: "Iniciando simulação..."
IA: "Simu;ações de Copas do Mundo: 10% completas"
IA: "Simu;ações de Copas do Mundo: 20% completas"
IA: "Simu;ações de Copas do Mundo: 30% completas"
IA: "Simu;ações de Copas do Mundo: 40% completas"
IA: "Simu;ações de Copas do Mundo: 50% completas"
IA: "Simu;ações de Copas do Mundo: 60% completas"
IA: "Simu;ações de Copas do Mundo: 70% completas"
IA: "Simu;ações de Copas do Mundo: 80% completas"
IA: "Simu;ações de Copas do Mundo: 90% completas"
IA: "Simu;ações de Copas do Mundo: 100% completas"
IA: "Simulação Finalizada!"


,1st,2nd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Brasil,0.603,0.230,0.126,0.041,0.833,0.566,0.345,0.199,0.117
Bélgica,0.567,0.262,0.127,0.044,0.829,0.505,0.309,0.184,0.108
Argentina,0.527,0.288,0.141,0.044,0.815,0.489,0.296,0.163,0.090
França,0.505,0.316,0.130,0.049,0.821,0.492,0.293,0.155,0.083
Inglaterra,0.432,0.277,0.179,0.112,0.709,0.473,0.247,0.136,0.077
Espanha,0.442,0.277,0.189,0.092,0.719,0.421,0.221,0.111,0.058
Uruguai,0.366,0.379,0.207,0.048,0.745,0.367,0.189,0.097,0.052
Holanda,0.547,0.272,0.133,0.048,0.819,0.439,0.216,0.108,0.049
Dinamarca,0.362,0.323,0.202,0.113,0.685,0.358,0.203,0.105,0.048


In [91]:
etapas=pd.DataFrame()
etapas['Cair na 1a Fase']=1-sim['Oitavas']
etapas['Cair nas Oitavas']=sim['Oitavas']-sim['Quartas']
etapas['Cair nas Quartas']=sim['Quartas']-sim['Semis']
etapas['Cair nas Semis']=sim['Semis']-sim['Final']
etapas['Cair na Final']=sim['Final']-sim['Campeão']
etapas['Ganhar a Final']=sim['Campeão']
etapas.to_excel('outputProbabilidadesPorEtapa.xlsx')
etapas

,Cair na 1a Fase,Cair nas Oitavas,Cair nas Quartas,Cair nas Semis,Cair na Final,Ganhar a Final
Seleção,,,,,,
Brasil,0.148,0.290,0.211,0.130,0.075,0.146
Bélgica,0.199,0.282,0.201,0.133,0.079,0.106
Argentina,0.176,0.329,0.186,0.138,0.067,0.104
Inglaterra,0.288,0.269,0.206,0.105,0.056,0.076
França,0.189,0.352,0.188,0.132,0.065,0.074
Espanha,0.283,0.304,0.188,0.093,0.067,0.065
Holanda,0.193,0.357,0.222,0.118,0.056,0.054
Portugal,0.214,0.369,0.198,0.101,0.067,0.051
Dinamarca,0.302,0.326,0.186,0.093,0.042,0.051


In [92]:
avanco=pd.DataFrame()
avanco['Avançar na 1a Fase']=sim['Oitavas']
avanco['Avançar nas Oitavas']=sim['Quartas']/sim['Oitavas']
avanco['Avançar nas Quartas']=sim['Semis']/sim['Quartas']
avanco['Avançar nas Semis']=sim['Final']/sim['Semis']
avanco['Avançar na Final']=sim['Campeão']/sim['Final']
avanco['Ganhar a Final']=sim['Campeão']
avanco.to_excel('outputAvançoPorEtapa.xlsx')
avanco

,Avançar na 1a Fase,Avançar nas Oitavas,Avançar nas Quartas,Avançar nas Semis,Avançar na Final,Ganhar a Final
Seleção,,,,,,
Brasil,0.852,0.659624,0.624555,0.629630,0.660633,0.146
Bélgica,0.801,0.647940,0.612717,0.581761,0.572973,0.106
Argentina,0.824,0.600728,0.624242,0.553398,0.608187,0.104
Inglaterra,0.712,0.622191,0.534989,0.556962,0.575758,0.076
França,0.811,0.565968,0.590414,0.512915,0.532374,0.074
Espanha,0.717,0.576011,0.544794,0.586667,0.492424,0.065
Holanda,0.807,0.557621,0.506667,0.482456,0.490909,0.054
Portugal,0.786,0.530534,0.525180,0.538813,0.432203,0.051
Dinamarca,0.698,0.532951,0.500000,0.500000,0.548387,0.051
